In [ ]:
from utils.parse_cif import decompress_files, load_structures
import numpy as np

import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
warnings.simplefilter('ignore', PDBConstructionWarning)

In [2]:
# Load Dev Data
dev_data_path = "data/dev/mmCIF"
#decompress_files(dev_data_path)

dev_data = load_structures(dev_data_path)
print(f"Loaded {len(dev_data)} structures from dev data")

Loaded structure: 2zq9
Loaded structure: 1h03
Loaded structure: 1sqk
Loaded structure: 5m1a
Failed to load 1p6g.cif: Empty file.
Loaded structure: 1p5k
Loaded structure: 1be3
Loaded structure: 4dg1
Failed to load 3j39.cif: Empty file.
Loaded structure: 1g5g
Loaded structure: 5gpe
Loaded structure: 1kyc
Loaded structure: 1a4e
Loaded structure: 1srq
Loaded structure: 1sfc
Loaded structure: 1hqj
Loaded structure: 2akw
Loaded structure: 4k6h
Loaded structure: 4i5q
Loaded structure: 1tye
Loaded structure: 6in5
Loaded structure: 1n0a
Loaded structure: 3sr6
Loaded structure: 1cvq
Loaded structure: 2x1w
Loaded structure: 5vqy
Loaded structure: 5s7w
Loaded structure: 1ibo
Loaded structure: 4zu5
Loaded structure: 3tmt
Loaded structure: 5bxn
Loaded structure: 3bwq
Loaded structure: 1o1d
Loaded structure: 3sn9
Loaded structure: 1is7
Loaded structure: 1w4k
Loaded structure: 1fmh
Loaded structure: 2f3i
Loaded structure: 5vmg
Loaded structure: 1mej
Loaded structure: 1jlc
Loaded structure: 4hlz
Loaded

In [19]:
def chain_to_ca_array(chain):
    """
    Convert a chain to a numpy array of C-alpha coordinates. Pass where there is no C-alpha atom.

    :param chain: A Bio.PDB Chain object
    :return: A numpy array of C-alpha coordinates.
    """
    ca_coords = []
    for residue in chain:
        if residue.has_id('CA'):
            ca_coords.append(residue['CA'].get_coord())
        else:
            pass
    return np.array(ca_coords)

first_chain = next(dev_data['1a4e'].get_chains(), None)
chain_to_ca_array(first_chain)

array([[112.052,  24.61 ,  60.097],
       [112.852,  21.664,  62.415],
       [109.909,  19.714,  63.792],
       ...,
       [162.743,  42.765,  42.227],
       [161.779,  40.665,  39.166],
       [165.453,  39.868,  38.508]], shape=(488, 3), dtype=float32)

In [ ]:
def ca_array_to_distances_features(ca_array: np.array):
    """
    Convert a C-alpha array to a 3d tensor of distance between C-alpha atoms.

    There are 6 features for each residue:
    1. d(ca[i-2], ca[i])
    2. d(ca[i], ca[i+2])
    3. d(ca[i-1], ca[i+1])
    4. d(ca[i-1], ca[i+2])
    5. d(ca[i-2], ca[i+2])
    6. d(ca[i-2], ca[i+1])

    The non applicable fields for two ends of chain are set to 0.


    :param ca_array: A numpy array of C-alpha coordinates.

    :return: A 3d tensor of distances.
    """

    n_residues = ca_array.shape[0]
    n_features = 6
    distances = np.zeros((n_residues, n_features))

    for i in range(n_residues):
        if i - 2 >= 0:
            distances[i, 0] = np.linalg.norm(ca_array[i] - ca_array[i - 2])
        if i + 2 < n_residues:
            distances[i, 1] = np.linalg.norm(ca_array[i] - ca_array[i + 2])
        if i - 1 >= 0 and i + 1 < n_residues:
            distances[i, 2] = np.linalg.norm(ca_array[i - 1] - ca_array[i + 1])
        if i - 1 >= 0 and i + 2 < n_residues:
            distances[i, 3] = np.linalg.norm(ca_array[i - 1] - ca_array[i + 2])
        if i - 2 >= 0 and i + 2 < n_residues:
            distances[i, 4] = np.linalg.norm(ca_array[i - 2] - ca_array[i + 2])
        if i - 2 >= 0 and i + 1 < n_residues:
            distances[i, 5] = np.linalg.norm(ca_array[i - 2] - ca_array[i + 1])

    return distances

# Test the function
first_chain = next(dev_data['1a4e'].get_chains(), None)
ca_array = chain_to_ca_array(first_chain)
distances = ca_array_to_distances_features(ca_array)
print(f"Distances shape: {distances.shape}")
print(f"Distances: {distances}")

Distances shape: (488, 6)
Distances: [[ 0.          6.49740791  0.          0.          0.          0.        ]
 [ 0.          6.31700039  6.49740791  9.14943886  0.          0.        ]
 [ 6.49740791  5.7420125   6.31700039  8.68606758 12.01934433  9.14943886]
 ...
 [ 5.52191687  5.43762016  5.51345444  5.31999636  6.08208704  4.97870541]
 [ 5.51345444  0.          5.43762016  0.          0.          5.31999636]
 [ 5.43762016  0.          0.          0.          0.          0.        ]]


In [22]:
def ca_array_to_angle_features(ca_array: np.array):
    """
    Convert a C-alpha array to a 3d tensor of angles between C-alpha atoms.

    There are 6 features for each residue:
    1. angle(ca[i-2], ca[i], ca[i+2])
    2. angle(ca[i-1], ca[i], ca[i+1])
    3. angle(ca[i-1], ca[i], ca[i-2])
    4. angle(ca[i], ca[i-1], ca[i-2])
    5. angle(ca[i], ca[i+1], ca[i+2])
    6. angle(ca[i+1], ca[i], ca[i+2])

    The non applicable fields for two ends of chain are set to 0.

    :param ca_array: A numpy array of C-alpha coordinates.

    :return: A 3d tensor of angles in radians.
    """
    n_residues = ca_array.shape[0]
    n_features = 6
    angles = np.zeros((n_residues, n_features))

    for i in range(n_residues):
        if i - 2 >= 0 and i + 2 < n_residues:
            angles[i, 0] = np.arccos(np.clip(np.dot(ca_array[i - 2] - ca_array[i], ca_array[i + 2] - ca_array[i]) /
                                              (np.linalg.norm(ca_array[i - 2] - ca_array[i]) *
                                               np.linalg.norm(ca_array[i + 2] - ca_array[i])), -1.0, 1.0))
        if i - 1 >= 0 and i + 1 < n_residues:
            angles[i, 1] = np.arccos(np.clip(np.dot(ca_array[i - 1] - ca_array[i], ca_array[i + 1] - ca_array[i]) /
                                              (np.linalg.norm(ca_array[i - 1] - ca_array[i]) *
                                               np.linalg.norm(ca_array[i + 1] - ca_array[i])), -1.0, 1.0))
        if i - 2 >= 0 and i + 1 < n_residues:
            angles[i, 2] = np.arccos(np.clip(np.dot(ca_array[i - 2] - ca_array[i], ca_array[i + 1] - ca_array[i]) /
                                              (np.linalg.norm(ca_array[i - 2] - ca_array[i]) *
                                               np.linalg.norm(ca_array[i + 1] - ca_array[i])), -1.0, 1.0))
        if i >= 2:
            angles[i, 3] = np.arccos(np.clip(np.dot(ca_array[i] - ca_array[i - 1], ca_array[i - 2] - ca_array[i]) /
                                              (np.linalg.norm(ca_array[i] - ca_array[i - 1]) *
                                               np.linalg.norm(ca_array[i - 2] - ca_array[i])), -1.0, 1.0))
        if i + 2 < n_residues:
            angles[i, 4] = np.arccos(np.clip(np.dot(ca_array[i] - ca_array[i + 1], ca_array[i + 2] - ca_array[i]) /
                                              (np.linalg.norm(ca_array[i] - ca_array[i + 1]) *
                                               np.linalg.norm(ca_array[i + 2] - ca_array[i])), -1.0, 1.0))
        if i + 1 < n_residues and i + 2 < n_residues:
            angles[i, 5] = np.arccos(np.clip(np.dot(ca_array[i + 1] - ca_array[i], ca_array[i + 2] - ca_array[i]) /
                                              (np.linalg.norm(ca_array[i + 1] - ca_array[i]) *
                                               np.linalg.norm(ca_array[i + 2] - ca_array[i])), -1.0, 1.0))
    return angles

# Test the function
first_chain = next(dev_data['1a4e'].get_chains(), None)
ca_array = chain_to_ca_array(first_chain)
angles = ca_array_to_angle_features(ca_array)
print(f"Angles shape: {angles.shape}")
print(f"Angles: {angles}")

Angles shape: (488, 6)
Angles: [[0.         0.         0.         0.         2.5948565  0.54673606]
 [0.         2.0411067  0.         0.         2.54017758 0.60141504]
 [2.76101542 1.94875836 2.13461828 2.5878427  2.418051   0.72354174]
 ...
 [1.17653728 1.6105634  1.06792426 2.38054037 2.36359763 0.77799493]
 [0.         1.5807811  1.16373098 2.379071   0.         0.        ]
 [0.         0.         0.         2.35877609 0.         0.        ]]


In [24]:
def ca_array_to_dihedral_features(ca_array: np.array):
    """
    Convert a C-alpha array to a 3d tensor of dihedral angles between C-alpha atoms.

    There are 6 features for each residue:
    1. dihedral(ca[i-2], ca[i-1], ca[i], ca[i+1])
    2. dihedral(ca[i-1], ca[i], ca[i+1], ca[i+2])

    The non applicable fields for two ends of chain are set to 0.

    :param ca_array: A numpy array of C-alpha coordinates.

    :return: A 3d tensor of angles in radians.
    """
    n_residues = ca_array.shape[0]
    n_features = 2
    dihedrals = np.zeros((n_residues, n_features))

    for i in range(n_residues):
        if i - 2 >= 0 and i + 1 < n_residues:
            b1 = ca_array[i - 2] - ca_array[i - 1]
            b2 = ca_array[i] - ca_array[i - 1]
            b3 = ca_array[i + 1] - ca_array[i]
            normal1 = np.cross(b1, b2)
            normal2 = np.cross(b2, b3)
            dihedrals[i, 0] = np.arctan2(np.linalg.norm(np.cross(normal1, normal2)), np.dot(normal1, normal2))
        if i - 1 >= 0 and i + 2 < n_residues:
            b1 = ca_array[i - 1] - ca_array[i]
            b2 = ca_array[i + 1] - ca_array[i]
            b3 = ca_array[i + 2] - ca_array[i + 1]
            normal1 = np.cross(b1, b2)
            normal2 = np.cross(b2, b3)
            dihedrals[i, 1] = np.arctan2(np.linalg.norm(np.cross(normal1, normal2)), np.dot(normal1, normal2))

    return dihedrals

# Test the function
first_chain = next(dev_data['1a4e'].get_chains(), None)
ca_array = chain_to_ca_array(first_chain)
dihedrals = ca_array_to_dihedral_features(ca_array)
print(f"Dihedrals shape: {dihedrals.shape}")
print(f"Dihedrals: {dihedrals}")


Dihedrals shape: (488, 2)
Dihedrals: [[0.         0.        ]
 [0.         1.10266101]
 [1.10266101 0.92660099]
 [0.92660099 2.23430896]
 [2.23430896 0.71916342]
 [0.71916342 0.1262268 ]
 [0.1262268  0.86434507]
 [0.86434507 0.19470686]
 [0.19470686 1.24402463]
 [1.24402463 1.83369529]
 [1.83369529 2.27023149]
 [2.27023149 1.3543725 ]
 [1.3543725  0.60450882]
 [0.60450882 1.22568572]
 [1.22568572 0.66985327]
 [0.66985327 0.32547531]
 [0.32547531 2.76508522]
 [2.76508522 0.83434808]
 [0.83434808 2.79245496]
 [2.79245496 0.85675508]
 [0.85675508 0.12091335]
 [0.12091335 1.10958624]
 [1.10958624 0.56421232]
 [0.56421232 1.76653206]
 [1.76653206 2.67957735]
 [2.67957735 2.16326427]
 [2.16326427 2.76872635]
 [2.76872635 1.23734522]
 [1.23734522 2.22116518]
 [2.22116518 1.34238887]
 [1.34238887 0.95268202]
 [0.95268202 0.73290098]
 [0.73290098 1.88117754]
 [1.88117754 1.379408  ]
 [1.379408   0.25823423]
 [0.25823423 2.04465342]
 [2.04465342 2.27865386]
 [2.27865386 2.35186791]
 [2.35186791 

In [25]:
def ca_array_to_proximity_features(ca_array: np.array):
    """
    Number and average distance and average residue distance of neighbors within a cutoff of 4.5A, 8A, 12A.

    Features:
    1. Number of neighbors within 4.5A
    2. Number of neighbors within 8A
    3. Number of neighbors within 12A
    4. Average distance of neighbors within 4.5A
    5. Average distance of neighbors within 8A
    6. Average distance of neighbors within 12A
    7. Average residue distance of neighbors within 4.5A
    8. Average residue distance of neighbors within 8A
    9. Average residue distance of neighbors within 12A

    :param ca_array: A numpy array of C-alpha coordinates.
    :return: A 3d tensor of distances.
    """
    n_residues = ca_array.shape[0]
    n_features = 9
    proximity = np.zeros((n_residues, n_features))

    cutoff = [4.5, 8.0, 12.0]

    for i in range(n_residues):
        for j in range(n_residues):
            if i != j:
                dist = np.linalg.norm(ca_array[i] - ca_array[j])
                for k in range(len(cutoff)):
                    if dist < cutoff[k]:
                        proximity[i, k] += 1
                        proximity[i, k + 3] += dist
                        proximity[i, k + 6] += np.abs(i - j)

    for i in range(n_residues):
        for k in range(len(cutoff)):
            if proximity[i, k] > 0:
                proximity[i, k + 3] /= proximity[i, k]
                proximity[i, k + 6] /= proximity[i, k]

    return proximity

# Test the function
first_chain = next(dev_data['1a4e'].get_chains(), None)
ca_array = chain_to_ca_array(first_chain)
proximity = ca_array_to_proximity_features(ca_array)
print(f"Proximity shape: {proximity.shape}")
print(f"Proximity: {proximity}")

Proximity shape: (488, 9)
Proximity: [[  1.           2.           6.         ...   1.           1.5
  103.66666667]
 [  2.           4.           8.         ...   1.           2.
    3.625     ]
 [  2.           6.           9.         ...   1.           2.16666667
    3.77777778]
 ...
 [  2.          10.          24.         ...   1.          14.7
   20.75      ]
 [  3.          10.          25.         ...  11.66666667  16.6
   20.48      ]
 [  1.           5.          19.         ...   1.           8.
   20.42105263]]


In [ ]:
def chain_to_features(chain: str):
    """
    Convert a chain to a 3d tensor of 23 features.
    the list of features is:
    1. distance(ca[i-2], ca[i])
    2. distance(ca[i], ca[i+2])
    3. distance(ca[i-1], ca[i+1])
    4. distance(ca[i-1], ca[i+2])
    5. distance(ca[i-2], ca[i+2])
    6. distance(ca[i-2], ca[i+1])
    7. angle(ca[i-2], ca[i], ca[i+2])
    8. angle(ca[i-1], ca[i], ca[i+1])
    9. angle(ca[i-1], ca[i], ca[i-2])
    10. angle(ca[i], ca[i-1], ca[i-2])
    11. angle(ca[i], ca[i+1], ca[i+2])
    12. angle(ca[i+1], ca[i], ca[i+2])
    13. dihedral(ca[i-2], ca[i-1], ca[i], ca[i+1])
    14. dihedral(ca[i-1], ca[i], ca[i+1], ca[i+2])
    15. Number of neighbors within 4.5A
    16. Number of neighbors within 8A
    17. Number of neighbors within 12A
    18. Average distance of neighbors within 4.5A
    19. Average distance of neighbors within 8A
    20. Average distance of neighbors within 12A
    21. Average residue distance of neighbors within 4.5A
    22. Average residue distance of neighbors within 8A
    23. Average residue distance of neighbors within 12A

    :param chain: A Bio.PDB Chain object
    :return: A 3d tensor of features.
    """
    ca_array = chain_to_ca_array(chain)
    distances = ca_array_to_distances_features(ca_array)
    angles = ca_array_to_angle_features(ca_array)
    dihedrals = ca_array_to_dihedral_features(ca_array)
    proximity = ca_array_to_proximity_features(ca_array)

    features = np.concatenate((distances, angles, dihedrals, proximity), axis=1)
    return features

# Test the function
first_chain = next(dev_data['1a4e'].get_chains(), None)
features = chain_to_features(first_chain)
print(f"Features shape: {features.shape}")
print(f"Features: {features}")

Features shape: (488, 23)
Features: [[  0.           6.49740791   0.         ...   1.           1.5
  103.66666667]
 [  0.           6.31700039   6.49740791 ...   1.           2.
    3.625     ]
 [  6.49740791   5.7420125    6.31700039 ...   1.           2.16666667
    3.77777778]
 ...
 [  5.52191687   5.43762016   5.51345444 ...   1.          14.7
   20.75      ]
 [  5.51345444   0.           5.43762016 ...  11.66666667  16.6
   20.48      ]
 [  5.43762016   0.           0.         ...   1.           8.
   20.42105263]]


In [1]:
from p2nd.utils.parse_cif import load_structure
from p2nd.utils.dssp import model_to_dssp, dssp_to_chain_labels
from p2nd.core.features import chain_to_features

# test the flow
file_path = "data/dev/mmCIF/1a4e.cif"

structure = load_structure(file_path)
model = structure[0]
chain = next(model.get_chains(), None)
features = chain_to_features(chain)
dssp = model_to_dssp(model, in_file=file_path)
labels = dssp_to_chain_labels(dssp, "A")

print(f"Features shape: {features.shape}")
print(f"Labels shape: {len(labels)}")

Loaded structure: 1a4e


FileNotFoundError: [Errno 2] No such file or directory: 'dssp'

In [2]:
# write features np array to file
import numpy as np

np.savetxt("features.txt", features, delimiter=",")